### Explainability

#### Introduction

Machine learning models are used to make decisions which should be accurate and explainable to understand what's going on.

Model explainers use statistical techniques to calculate _feature importance_. Explainers work by evaluating a test data set of feature cases and the labels the model predicts for them.

#### Globally vs. Locally

This assesment can be done globally or locally: the former concerns the relationship between predictions and features, the latter focuses on a single prediction.

For the local case, for each label, if we are talking about a classification setting, the feature importance has a value for each label (also for the multi-class classification case). To make a decision, the overall feature importance is computed and the highest will determine the final choice.
It's noteworthy that the global result can differ from the local results.

For regression models, feature importance will tell you the level of influence each feature has on the predicted scalar label.

#### Development

Azure Experiments aren't suited for training explainers but the following method sould be used:
- _MimicExplainer_ which tries to explain a model using _a global surrogate model_ characterized by the same architecture of the original one. You should select the most similar architecture to the original one for the surrogate explainer.
- _TabularExplainer_ exploits a wrapper around SHAP explainer algorithms, automatically choosing the one that is most appropriate for your model architecture.
- _PFIExplainer_ (Permutation Feature Importance) uses shuffle procedures for measuring the impace on predictions.

### Explain a local model

In [ ]:
# Define an experiment.

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

In [6]:
# Let's start with a Tabular Explainer.
from interpret.ext.blackbox import TabularExplainer

tab_explainer = TabularExplainer(model, X_train, features = features, classes = labels)

print(tab_explainer, 'ready!')

In [ ]:
# Global feature importance.

global_tab_explanation = tab_explainer.explain_global(X_train)
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()

for feature, importance in global_tab_feature_importance:
    print(feature, ':', importance)

In [ ]:
# Local explanation.

X_explain = X_test[0:2]
predictions = model.predict(X_explain)

local_tab_explanation = tab_explainer.explain_local(X_explain)

local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_names()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

### Explain a registered model

In [17]:
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ComputeTarget

ws = Workspace.from_config()
env = Environment.from_conda_specification('experiment_env', 'environment.yml')
compute = ComputeTarget(workspace=ws, name = 'ravazzil-compute')

In [13]:
# After training a model, a TabularExplainer is used to explain the model
# and ExplanationClient upload it to the output folder.

In [18]:
from azureml.core import ScriptRunConfig
from azureml.core import Experiment

run_config = ScriptRunConfig(source_directory = './Script',
                             script='11_Explainability.py',
                             environment=env,
                             compute_target=compute
                             )
exp = Experiment(name = 'explain-exp', workspace = ws)
run = exp.submit(run_config)
run.wait_for_completion()

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_5518ccf8fcac4a509c5f671e21707be5/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"<string>\", line 197, in <module>\n  File \"<string>\", line 193, in main\n  File \"/azureml-envs/azureml_5518ccf8fcac4a509c5f671e21707be5/lib/python3.6/runpy.py\", line 261, in run_path\n    code, fname = _get_code_from_file(run_name, path_name)\n  File \"/azureml-envs/azureml_5518ccf8fcac4a509c5f671e21707be5/lib/python3.6/runpy.py\", line 236, in _get_code_from_file\n    code = compile(f.read(), fname, 'exec')\n  File \"11_Explainability.py\", line 1\n    %%writefile $experiment_folder/diabetes_training.py\n    ^\nSyntaxError: invalid syntax\n\n",
        "messageParameters": {},
        "details": []
   

In [ ]:
# Retrieve the results.

from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)